In [161]:
from google.colab import files
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import torch
import torchvision
from torch import optim
from torch import nn
from torch.utils.data import Dataset
import random
import math

import necessary libraries

In [162]:
files.upload()
os.system("pip install kaggle")
os.system("mkdir -p ~/.kaggle")
os.system("cp kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")
!kaggle datasets download -d miladfa7/persian-wikipedia-dataset
!unzip /content/persian-wikipedia-dataset.zip

Saving kaggle.json to kaggle (2).json
persian-wikipedia-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/persian-wikipedia-dataset.zip
replace Persian-WikiText-1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Persian-WikiText-1.txt  
  inflating: Persian-WikiText-2.txt  
  inflating: Persian-WikiText-3.txt  
  inflating: Persian-WikiText-4.txt  
  inflating: Persian-WikiText-5.txt  
  inflating: Persian-WikiText-6.txt  
  inflating: Persian-WikiText-7.txt  
  inflating: Persian-WikiText-8.txt  
  inflating: Persian-WikiText-9.txt  


import data files from kaggle

In [163]:
sentences1 = open("/content/Persian-WikiText-1.txt", encoding = 'utf8').read().replace("\n","").split('.')
sentences2 = open("/content/Persian-WikiText-2.txt", encoding = 'utf8').read().replace("\n","").split('.')
sentences3 = open("/content/Persian-WikiText-3.txt", encoding = 'utf8').read().replace("\n","").split('.')
sentences4 = open("/content/Persian-WikiText-4.txt", encoding = 'utf8').read().replace("\n","").split('.')
sentences5 = open("/content/Persian-WikiText-5.txt", encoding = 'utf8').read().replace("\n","").split('.')
sentences6 = open("/content/Persian-WikiText-6.txt", encoding = 'utf8').read().replace("\n","").split('.')
sentences7 = open("/content/Persian-WikiText-7.txt", encoding = 'utf8').read().replace("\n","").split('.')
sentences8 = open("/content/Persian-WikiText-8.txt", encoding = 'utf8').read().replace("\n","").split('.')
sentences9 = open("/content/Persian-WikiText-9.txt", encoding = 'utf8').read().replace("\n","").split('.')
sentences = sentences1 +sentences2 + sentences3 + sentences4 + sentences5 + sentences6 + sentences7 + sentences8 + sentences9

تمام نه فایل موجود را باز کرده، جملات آن را جدا می کنیم و در انتها تمام جملات را داخل یک آرایه میگذاریم. ولیکن قبل از آن تمام جملاتی که در چند خط هستند را به یک خط می آوریم و اینکه جمله ها را از طریق نقاط پایان جمله شناسایی میکینیم

In [164]:
#######################################################
#               Define Vocabulary Class
#######################################################

class Vocabulary:
  
    
    def __init__(self, freq_threshold, max_size):
        '''
        freq_threshold : the minimum times a word must occur in corpus to be treated in vocab
        max_size : max source vocab size. Eg. if set to 10,000, we pick the top 10,000 most frequent words and discard others
        '''
        #initiate the index to token dict
        ## <PAD> -> padding, used for padding the shorter sentences in a batch to match the length of longest sentence in the batch
        ## <SOS> -> start token, added in front of each sentence to signify the start of sentence
        ## <EOS> -> End of sentence token, added to the end of each sentence to signify the end of sentence
        ## <UNK> -> words which are not found in the vocab are replace by this token
        self.itos = {0: '<PAD>', 1:'<SOS>', 2:'<EOS>', 3: '<UNK>'}
        #initiate the token to index dict
        self.stoi = {k:j for j,k in self.itos.items()} 
        
        self.freq_threshold = freq_threshold
        self.max_size = max_size
    
    '''
    __len__ is used by dataloader later to create batches
    '''
    def __len__(self):
        return len(self.itos)
    
    '''
    a simple tokenizer to split on space and converts the sentence to list of words
    '''
    @staticmethod
    def tokenizer(text):
        return [tok.lower().strip() for tok in text.split(' ')]
    
    '''
    build the vocab: create a dictionary mapping of index to string (itos) and string to index (stoi)
    output ex. for stoi -> {'the':5, 'a':6, 'an':7}
    '''
    def build_vocabulary(self, sentence_list):
        #calculate the frequencies of each word first to remove the words with freq < freq_threshold
        frequencies = {}  #init the freq dict
        idx = 4 #index from which we want our dict to start. We already used 4 indexes for pad, start, end, unk
        
        #calculate freq of words
        for sentence in sentence_list:
            for word in self.tokenizer(sentence):
                if word not in frequencies.keys():
                    frequencies[word]=1
                else:
                    frequencies[word]+=1
                    
                    
        #limit vocab by removing low freq words
        frequencies = {k:v for k,v in frequencies.items() if v>self.freq_threshold} 
        
        #limit vocab to the max_size specified
        frequencies = dict(sorted(frequencies.items(), key = lambda x: -x[1])[:self.max_size-idx]) # idx =4 for pad, start, end , unk
            
        #create vocab
        for word in frequencies.keys():
            self.stoi[word] = idx
            self.itos[idx] = word
            idx+=1
            
    '''
    convert the list of words to a list of corresponding indexes
    '''    
    def numericalize(self, text):
        #tokenize text
        tokenized_text = self.tokenizer(text)
        numericalized_text = []
        for token in tokenized_text:
            if token in self.stoi.keys():
                numericalized_text.append(self.stoi[token])
            else: #out-of-vocab words are represented by UNK token index
                numericalized_text.append(self.stoi['<UNK>'])
                
        return numericalized_text

تعریف کلاس دیکشنری

In [165]:
first_of_sentences = []
rest_of_the_sentences = []
count = 0
for sentence in sentences:
  if count == 1000:
    break
  count += 1
  splitted_sentence = sentence.split()
  first_of_sentences.append(splitted_sentence[0:5])
  rest_of_the_sentences.append(splitted_sentence[5:])
firsts = []
for item in first_of_sentences:
  firsts.append(" ".join(item))
rests = []
for item in rest_of_the_sentences:
  rests.append(" ".join(item))

در ادامه برای ساخت دیتا ست، ما دو تا آرایه میسازیم. آرایه اول بدین صورت ساخته میشود که 5 کلمه ی ابتدای هر جمله را به عنوان یک عضو آرایه میگذاریم و آرایه دوم بدین صورت ساخته میشود که بقیه هر جمله ( به غیر از پنج کلمه اول ) را در آن میگذاریم. گفتنی است که مثلا عضو اول از آرایه اول و عضو اول از آرایه دوم مکمل یکدیگر هستند

In [166]:
voc = Vocabulary(2, 10000)
voc.build_vocabulary(sentences)

ساخت دیکشنری از روی تمام جملات

In [167]:
raw_data = {
    'firsts': [first for first in firsts],
    'rests': [rest for rest in rests]
}

df = pd.DataFrame(raw_data, columns = ['firsts', 'rests'])

train = pd.DataFrame({
    'firsts': df['firsts'][0:800],
    'rests': df['rests'][0:800]
})

test = pd.DataFrame({
    'firsts': df['firsts'][800:1000],
    'rests': df['rests'][800:1000]
})


یک دیتا فریم پانداز میسازیم که آرایه اول در قسمت های قبلی را در کنار آرایه دوم قرار دهد یعنی داده ورودی اعضای آرایه اول باشند و اعضای آرایه دوم لیبل های مربوط به ورودی ها باشند. در ضمن در این مرحله داده آموزش و تست را نیز جدا میکنیم

In [168]:

#######################################################
#               Define Train_Dataset class
#######################################################

class Train_Dataset(Dataset):
    '''
    Initiating Variables
    df: the training dataframe
    source_column : the name of source text column in the dataframe
    target_columns : the name of target text column in the dataframe
    transform : If we want to add any augmentation
    '''
    
    def __init__(self, df, source_column, target_column, voc, transform=None):
    
        self.df = df
        self.transform = transform
        
        #get source and target texts
        self.source_texts = self.df[source_column]
        self.target_texts = self.df[target_column]
        
        
        self.vocab = voc
        
    def __len__(self):
        return len(self.df)
    
    '''
    __getitem__ runs on 1 example at a time. Here, we get an example at index and return its numericalize source and
    target values using the vocabulary objects we created in __init__
    '''
    def __getitem__(self, index):
        source_text = self.source_texts[index]
        target_text = self.target_texts[index]
        
        if self.transform is not None:
            source_text = self.transform(source_text)
            
        #numericalize texts ['<SOS>','cat', 'in', 'a', 'bag','<EOS>'] -> [1,12,2,9,24,2]
        numerialized_source = [self.vocab.stoi["<SOS>"]]
        numerialized_source += self.vocab.numericalize(source_text)
        numerialized_source.append(self.vocab.stoi["<EOS>"])
    
        numerialized_target = [self.vocab.stoi["<SOS>"]]
        numerialized_target += self.vocab.numericalize(target_text)
        numerialized_target.append(self.vocab.stoi["<EOS>"])
        
        #convert the list to tensor and return
        return torch.tensor(numerialized_source), torch.tensor(numerialized_target)

در اینجا دیتا ست را تعریف میکنیم به گونه ای که  جملات داخل دیتا فریم پانداز به کلمه ها تقسیم شوند و هر کلمه متناظر با یک عدد در دیکشنری شود

In [169]:
train_dataset = Train_Dataset(train, 'firsts', 'rests', voc)
for i in range(5):
  print(train_dataset[i])

(tensor([   1,   15,   17, 4751,    3,    3,    2]), tensor([   1,    3,   17, 1610,    3, 5830,    3, 1422,    6,   71,    3,    5,
           3,   18, 4127, 9830,    3, 1794,   31, 1252,   16, 2856,  456,    5,
         521,  453,    9,   10,   16,  521,  318, 6380,  273,   11,   29,    5,
          67,  806,   10,    6, 1748,    5, 1252, 1112,   77,   45,   11,  172,
        2557,   19,   13, 6945,    5, 3385,   41,    2]))
(tensor([   1,   38, 1610, 5830,  700,   30,    2]), tensor([   1, 1770,    9,   10,    8,  700,   14, 1610,    3,    3,    5,    3,
           3,    3,   35,    3,  126,   20,    9,    2]))
(tensor([   1,  470, 1610, 5830, 5476,    5,    2]), tensor([   1,  630,  215,   18, 4127,   16, 2856,  456,    6,  789,   88,   14,
           3,  388,    9,    2]))
(tensor([   1, 1610, 2544,  483,    4,   81,    2]), tensor([   1,  560,  655, 1704,    3,  283,    3,    6,   71,    3,   24,    3,
        1964,    3,  273,   23,    2]))
(tensor([   1, 1209, 3393,   19,    3,

دیتاست را میسازیم

In [170]:

#######################################################
#               Define Dataset Class
#######################################################

class Validation_Dataset:
    def __init__(self, train_dataset, df, source_column, target_column, transform = None):
        self.df = df
        self.transform = transform
        
        #train dataset will be used as lookup for vocab
        self.train_dataset = train_dataset
        
        #get source and target texts
        self.source_texts = self.df[source_column]
        self.target_texts = self.df[target_column]
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        source_text = self.source_texts[index]
        #print(source_text)
        target_text = self.target_texts[index]
        #print(target_text)
        if self.transform is not None:
            source_text = self.transform(source_text)
            
        #numericalize texts ['<SOS>','cat', 'in', 'a', 'bag','<EOS>'] -> [1,12,2,9,24,2]
        numerialized_source = [self.train_dataset.vocab.stoi["<SOS>"]]
        numerialized_source += self.train_dataset.vocab.numericalize(source_text)
        numerialized_source.append(self.train_dataset.vocab.stoi["<EOS>"])
    
        numerialized_target = [self.train_dataset.vocab.stoi["<SOS>"]]
        numerialized_target += self.train_dataset.vocab.numericalize(target_text)
        numerialized_target.append(self.train_dataset.vocab.stoi["<EOS>"])
        #print(numerialized_source)
        return torch.tensor(numerialized_source), torch.tensor(numerialized_target)


دیتاست مربوط به داده های تست را تعریف میکینم

In [171]:
test_dataset = Validation_Dataset(train_dataset, df, 'firsts', 'rests')
for i in range(5):
  print(train_dataset[i])

(tensor([   1,   15,   17, 4751,    3,    3,    2]), tensor([   1,    3,   17, 1610,    3, 5830,    3, 1422,    6,   71,    3,    5,
           3,   18, 4127, 9830,    3, 1794,   31, 1252,   16, 2856,  456,    5,
         521,  453,    9,   10,   16,  521,  318, 6380,  273,   11,   29,    5,
          67,  806,   10,    6, 1748,    5, 1252, 1112,   77,   45,   11,  172,
        2557,   19,   13, 6945,    5, 3385,   41,    2]))
(tensor([   1,   38, 1610, 5830,  700,   30,    2]), tensor([   1, 1770,    9,   10,    8,  700,   14, 1610,    3,    3,    5,    3,
           3,    3,   35,    3,  126,   20,    9,    2]))
(tensor([   1,  470, 1610, 5830, 5476,    5,    2]), tensor([   1,  630,  215,   18, 4127,   16, 2856,  456,    6,  789,   88,   14,
           3,  388,    9,    2]))
(tensor([   1, 1610, 2544,  483,    4,   81,    2]), tensor([   1,  560,  655, 1704,    3,  283,    3,    6,   71,    3,   24,    3,
        1964,    3,  273,   23,    2]))
(tensor([   1, 1209, 3393,   19,    3,

دیتاست داده های تست را میسازیم

In [172]:

#######################################################
#            Define Dataloader Functions
#######################################################


def get_train_loader(dataset, batch_size):
    #define loader
    loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size) 
    return loader

def get_valid_loader(dataset, batch_size):
    #define loader
    loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size)
    return loader

In [173]:
train_loader = get_train_loader(train_dataset, 1)
test_loader = get_valid_loader(test_dataset, 1)

دیتا لودر ها را میسازیم

In [174]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, hid_data):
        
        #src.shape = [batch size, src len]
        
        embedded = self.dropout(self.embedding(src))
        
        outputs, hidden = self.rnn(embedded, hid_data) #no cell state!
        
        
        #outputs are always from the top hidden layer
        
        return hidden

تعریف انکودر

In [175]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = ورودی هر دفعه همان خروجی قبلی است که متناظر با یک کلمه است
        #hidden = انتقالی از زمان قبلی
        #context = latent_space
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        
        input = input.unsqueeze(0)
        
        
        embedded = self.dropout(self.embedding(input))
        
                
        emb_con = torch.cat((embedded, context), dim = 1)
            
            
        output, hidden = self.rnn(emb_con, hidden)
        
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        
        output = torch.cat((embedded, hidden, context), 
                           dim = 1)
        
        
        prediction = self.fc_out(output)
        
        
        return prediction, hidden

تعریف دیکودر

In [176]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [bach size, src len]
        #trg = [batch size, trg len]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        input_length = src.shape[1]
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        src = src[0,:]
        hid_list = []
        for j in range(512):
          hid_list.append(1)
        hid_data = torch.tensor([hid_list], device = self.device, dtype = torch.float32)
        for i in range(input_length):
           encoder_hidden = self.encoder(torch.tensor([src[i]], device = self.device), hid_data)
           hid_data = encoder_hidden
        context = encoder_hidden
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        trg = trg[0,:]
        input = trg[0]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1[0]

        return outputs

define Seq2Seq Class

In [177]:
INPUT_DIM = len(voc)
OUTPUT_DIM = len(voc)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)
model.to(device)

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10000, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10000, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=10000, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

تعیین پارامتر های مدل

In [178]:
optimizer = optim.Adam(model.parameters())
PAD_IDX = voc.stoi['<PAD>']

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

آماده سازی برای آموزش

In [179]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for first, rest in iterator:
        
        src = first
        src = src.to("cuda")
        trg = rest
        trg = trg.to("cuda")
        trg_len = trg.shape[1]
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        output.to(device)
        
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[0,:]
        trg = trg[0:trg_len - 1]
        
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [180]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for first, rest in iterator:

            src = first
            src = src.to("cuda")
            trg = rest
            trg = trg.to("cuda")
            trg_len = trg.shape[1]

            output = model(src, trg, 0) #turn off teacher forcing


            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[0,:]
            trg = trg[0:trg_len - 1]


            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

تعریف توابع آموزش و تست

In [181]:
N_EPOCHS = 10
CLIP = 1


for epoch in range(N_EPOCHS):
    
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    test_loss = evaluate(model, test_loader, criterion)
    
    
    
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Test. Loss: {test_loss:.3f} |  Test. PPL: {math.exp(test_loss):7.3f}')

Epoch: 01
	Train Loss: 5.475 | Train PPL: 238.576
	 Test. Loss: 5.595 |  Test. PPL: 269.146
Epoch: 02
	Train Loss: 4.153 | Train PPL:  63.610
	 Test. Loss: 5.743 |  Test. PPL: 312.029
Epoch: 03
	Train Loss: 3.648 | Train PPL:  38.380
	 Test. Loss: 5.682 |  Test. PPL: 293.391
Epoch: 04
	Train Loss: 3.301 | Train PPL:  27.152
	 Test. Loss: 5.878 |  Test. PPL: 357.187
Epoch: 05
	Train Loss: 3.062 | Train PPL:  21.368
	 Test. Loss: 6.154 |  Test. PPL: 470.804
Epoch: 06
	Train Loss: 2.839 | Train PPL:  17.096
	 Test. Loss: 6.344 |  Test. PPL: 568.989
Epoch: 07
	Train Loss: 2.648 | Train PPL:  14.130
	 Test. Loss: 6.317 |  Test. PPL: 553.726
Epoch: 08
	Train Loss: 2.531 | Train PPL:  12.563
	 Test. Loss: 7.164 |  Test. PPL: 1291.727
Epoch: 09
	Train Loss: 2.410 | Train PPL:  11.130
	 Test. Loss: 6.192 |  Test. PPL: 488.593
Epoch: 10
	Train Loss: 2.333 | Train PPL:  10.309
	 Test. Loss: 6.693 |  Test. PPL: 806.410


مدل را تنها بر روی 1000 جمله اموزش داده ایم و همانطور که میبینید مدل به سمت بیش تطابقی میرود

در ادامه مدل را بر روی ده داده تست میکنیم

In [190]:
count = 0
for data, label in train_loader:
  if(count == 10):
    break
  count += 1
  data = data.to(device)
  label = label.to(device)
  print("our first five words:", data)
  print("rest of the sentence:", label)
  print("\n")
  model.eval()
  output = model(data, label, 0)
  output_dim = output.shape[-1]
  output = output[1:].view(-1, output_dim)
  rest_of_the_sentence = []
  for i in range(output.shape[0]):
    word = torch.argmax(output[i])
    rest_of_the_sentence.append(word.tolist())
  print("our prediction is:", rest_of_the_sentence)
  print("this is for round:", count, "\n")

our first five words: tensor([[   1,   15,   17, 4751,    3,    3,    2]], device='cuda:0')
rest of the sentence: tensor([[   1,    3,   17, 1610,    3, 5830,    3, 1422,    6,   71,    3,    5,
            3,   18, 4127, 9830,    3, 1794,   31, 1252,   16, 2856,  456,    5,
          521,  453,    9,   10,   16,  521,  318, 6380,  273,   11,   29,    5,
           67,  806,   10,    6, 1748,    5, 1252, 1112,   77,   45,   11,  172,
         2557,   19,   13, 6945,    5, 3385,   41,    2]], device='cuda:0')


our prediction is: [1, 593, 2719, 8280, 11, 11, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
this is for round: 1 

our first five words: tensor([[   1,   38, 1610, 5830,  700,   30,    2]], device='cuda:0')
rest of the sentence: tensor([[   1, 1770,    9,   10,    8,  700,   14, 1610,    3,    3,    5,    3,
            3,    3,   35,    3,  126,   20,    9,    2]], device='cuda

به دلیل بیش انطباقی بر روی داده های تست آزمایش انجام دادیم. و میتوان توسط دیکشنری این آرایه ها را به حروف تبدیل کرد